# More Practice with Movies

In this assignment I introduce a third table in the movie database, the release_date table.  Thus in the postgresql database we now have three tables:

* moviecast
* title
* release_date

The tables have the following schema:

```
 Table "public.title"
 Column |  Type  | Modifiers 
--------+--------+-----------
 index  | bigint | 
 title  | text   | 
 year   | bigint | 


  Table "public.moviecast"
  Column   |       Type       | Modifiers 
-----------+------------------+-----------
 index     | bigint           | 
 title     | text             | 
 year      | bigint           | 
 name      | text             | 
 type      | text             | 
 character | text             | 
 n         | double precision | 



 Table "public.release_date"
 Column  |  Type  | Modifiers 
---------+--------+-----------
 index   | bigint | 
 title   | text   | 
 year    | bigint | 
 country | text   | 
 date    | date   | 
 month   | bigint |
 day     | bigint |
 dow     | bigint |
```


### Hints

* Warning, some of these queries can create very large cartesian products!   You may want to use query to help reduce the size of both relations.
* The ``njoin`` operator is more efficient with memory than cartesian product.  You should prefer njoin where you can.


In [21]:
from reframe import Relation
import warnings
warnings.filterwarnings('ignore')
from sols2 import *

moviecast = Relation('/home/faculty/millbr02/pub/cast.csv',sep=',')
title = Relation('/home/faculty/millbr02/pub/titles.csv',sep=',')
release_date = Relation('/home/faculty/millbr02/pub/release_dates.csv',sep=',')

In [22]:
%load_ext sql


The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [23]:
%sql postgresql://millbr02:@localhost/movies

'Connected: millbr02@movies'

In [24]:
import pandas as pd

from sqlalchemy import create_engine
engine = create_engine('postgresql://millbr02:@localhost/movies')

def len_check(act,got):
    if type(act) != list:
        act = [act]
    allres = [len(a) == len(got) for a in act]
    if not any(allres):
        ix = allres.index(False)
        print("Error, results are not the same length.  I was expecting {} rows but got {}".format(len(act[ix]),len(got)))
        assert False

def col_check(act,got):
    if list(act.columns) != list(got.columns):
        print("Error: I was expecting columns: ", list(act.columns))
        assert False

def show_diff(act,got):
    print("Found differences in two results:")
    diff = False
    for i in range(len(act)):
        if (act.iloc[i] != got.iloc[i]).all():
            diff = True
            print("diff in row", i, "expected: ", act.iloc[i])
            print("Got: ", got.iloc[i])
    if diff:
        assert False
        

def sol_cp1r(release_date,moviecast,got):
    act = release_date.query('year == 2014').groupby(['country']).count('year').rename('count_year','year')
    if list(act.columns) != list(got.columns):
        print("Error: I was expecting columns: ", list(act.columns))
        assert False
    len_check(act,got)
    if not got.sort(['country','year']).equals(act.sort(['country','year'])):
        show_diff(act.sort(['country','year']),got.sort(['country','year']))
        assert False

def sol_cp2r(release_date,moviecast,got):
    act = release_date.query('year > 1975').njoin(moviecast).query("name == 'Clint Eastwood' & n == 1").project(['country']).sort(['country'])
    if list(act.columns) != list(got.columns):
        print("Error: I was expecting columns: ", list(act.columns))
        assert False
    len_check(act,got)
    if not got.equals(act):
        show_diff(act,got)
        assert False
    
def sol_cp3r(release_date,moviecast,got):
    act = release_date.query("dow == 0 & year == 2014 & country == 'USA'").njoin(moviecast).query("n == 1").project(['name','title'])
    if list(act.columns) != list(got.columns):
        print("Error: I was expecting columns: ", list(act.columns))
        assert False
    len_check(act,got)
    if not got.sort(['name','title']).equals(act.sort(['name','title'])):
        show_diff(act.sort(['name','title']),got.sort(['name','title']))
        assert False

def sol_cp4r(release_date,moviecast,got):    
    act = release_date.query("month == 11 | month == 12 country == 'USA'").njoin(moviecast.query("n == 1 & year >= 2010")).groupby(['name']).count("title").sort(['count_title'],ascending=False).head(10)
    if list(act.columns) != list(got.columns):
        print("Error: I was expecting columns: ", list(act.columns))
        assert False
    len_check(act,got)
    if not got.equals(act):
        show_diff(act,got)
        assert False

def sol_cp5r(release_date,moviecast,got):
    act = release_date.query("(country == 'Germany' | country == 'USA') & year == 2014").\
    cartesian_product(release_date.query("(country == 'Germany' | country == 'USA') & year == 2014")).\
    query('date_x < date_y & country_x == "Germany" & country_y == "USA" & title_x == title_y').\
    project(['title_x']).rename('title_x','title')
    if list(act.columns) != list(got.columns):
        print("Error: I was expecting columns: ", list(act.columns))
        assert False
    len_check(act,got)
    if not got.sort(['title']).equals(act.sort(['title'])):
        show_diff(act.sort(['title']),got.sort(['title']))
        assert False

    
def to_df(rs):
    return pd.DataFrame(rs, columns=(rs and rs.keys) or [])

def sol_cp1s(got):
    q = "select country, count(*) from release_date where year = 2014 group by country"
    act = pd.read_sql_query(q,engine)
    got = to_df(got)
    col_check(act,got)
    len_check(act,got)
    if not got.sort(['country','count']).equals(act.sort(['country','count'])):
        show_diff(act.sort(['country','count']),got.sort(['country','count']))
        assert False
    
def sol_cp2s(got):
    q = '''select distinct country
from release_date natural join moviecast
where name = 'Clint Eastwood' and release_date.year > 1975 and n = 1
order by country
'''
    act = pd.read_sql_query(q,engine)
    got = to_df(got)
    col_check(act,got)
    len_check(act,got)
    if not got.equals(act):
        show_diff(act,got)
        assert False
    
def sol_cp3s(got):
    q = '''select name, moviecast.title
from moviecast natural join release_date
where dow = 0 and release_date.year = 2014 and country = 'USA' and n = 1
'''
    q2 = '''select distinct name, moviecast.title
from moviecast natural join release_date
where dow = 0 and release_date.year = 2014 and country = 'USA' and n = 1
'''
    act = pd.read_sql_query(q,engine)
    act2 = pd.read_sql_query(q2,engine)
    got = to_df(got)
    col_check(act,got)
    len_check([act,act2],got)
    if not got.sort(['name','title']).equals(act.sort(['name','title'])) or got.sort(['name','title']).equals(act2.sort(['name','title'])):
        show_diff(act.sort(['name','title']),got.sort(['name','title']))
        assert False

    
def sol_cp4s(got):
    q = '''select moviecast.name
from moviecast natural join release_date
where release_date.year >= 2010 and n = 1 and country = 'USA' and (month = 11 or month = 12)
group by name
order by count(*) desc
limit 10
'''
    act = pd.read_sql_query(q,engine)
    got = to_df(got)
    col_check(act,got)
    len_check(act,got)
    if not got.equals(act):
        show_diff(act,got)
        

    
def sol_cp5s(got):
    q = '''select A.title
from release_date A, release_date B
where A.country = 'Germany' and B.country = 'USA' and A.date < B.date and A.title = B.title and A.year = 2014 and B.year = 2014
'''
    act = pd.read_sql_query(q,engine)
    got = to_df(got)
    col_check(act,got)
    len_check(act,got)
    assert got.sort(['title']).equals(act.sort(['title']))



    


## 1.  How many movies were released in each country in the year 2014?

Your result should be a table that contains the columns country and year

In [25]:
release_date.query('year == 2014').groupby(['country']).count('year').project(['country','count_year'])

,country,count_year
0,Afghanistan,3
1,Albania,30
2,Algeria,3
3,Andorra,1
4,Angola,4
5,Antigua and Barbuda,1
6,Argentina,216
7,Armenia,16
8,Aruba,26
9,Australia,230


In [ ]:
sol_cp1r(release_date,moviecast,_)

In [27]:
%%sql

select country, count(year) from release_date where year = 2014 group by country;

152 rows affected.


country,count
Costa Rica,19
Cambodia,79
Turkey,316
Cyprus,57
Samoa,1
Slovenia,113
Vietnam,143
Kuwait,346
Jamaica,24
Antigua and Barbuda,1


In [ ]:
sol_cp1s(_)

## 2.  Show all of the countries in alphabetical order that released a movie starring clint eastwood after 1975

Your answer should be a single column with the names of the countries.  The country names should not be duplicated.

In [28]:
release_date.query('year > 1975').njoin(moviecast).query("name == 'Clint Eastwood' & n == 1").project(['country']).sort(['country'])

,country
404237,Argentina
404333,Australia
1435310,Austria
403901,Belgium
12261082,Bolivia
823231,Brazil
404261,Bulgaria
403877,Canada
3935447,Chile
1620196,Colombia


In [ ]:
sol_cp2r(release_date,moviecast,_)

In [29]:
%%sql

select distinct country from moviecast natural join release_date 
where year > 1975 and name = 'Clint Eastwood' and moviecast.n = 1
order by country;

65 rows affected.


country
Argentina
Australia
Austria
Belgium
Bolivia
Brazil
Bulgaria
Canada
Chile
Colombia


In [ ]:
sol_cp2s(_)

## 3.  Show the name of the lead actor/actress and title of the movie where the movie was released on a sunday in the USA during 2014.

In [30]:
release_date.query("dow == 0 & country == 'USA' & year == 2014").njoin(moviecast.query("n == 1")).sort(['name', 'title']).project(['name', 'title'])

,name,title
27,April Hollingsworth,Prosper
28,Ashley (VII) James,Rebound (III)
21,Barry Corbin,Mountain Top
36,Brandon Jacobs,The Grievance Group
37,Cabrina Collesides,The Grievance Group
8,Cameron Bender,Find Me
30,Christopher (VI) Hunt,Right to Believe
10,Daniel (II) Gilchrist,Ghost in da hood
15,Dean Cain,Holiday Miracle
46,Donnie Yen,Xi you ji: Da nao tian gong


In [ ]:
sol_cp3r(release_date,moviecast,_)

In [31]:
%%sql

select distinct name, title
from moviecast natural join release_date
where n = 1 and release_date.year = 2014 and dow = 0 and country = 'USA'
order by name, title;

42 rows affected.


name,title
April Hollingsworth,Prosper
Ashley (VII) James,Rebound (III)
Barry Corbin,Mountain Top
Brandon Jacobs,The Grievance Group
Cabrina Collesides,The Grievance Group
Cameron Bender,Find Me
Christopher (VI) Hunt,Right to Believe
Daniel (II) Gilchrist,Ghost in da hood
Dean Cain,Holiday Miracle
Donnie Yen,Xi you ji: Da nao tian gong


In [ ]:
sol_cp3s(_)

## 4. Show the top 10 actors/actresses that have starred in the most movies released in the USA in the christmas season since (and including) the year 2010

Lets define the Christmas season as the months of November and December.

In [32]:
release_date.query("(month == 11 | month == 12) & country == 'USA'").njoin(moviecast.query("n==1 & year >= 2010")).\
groupby(['name']).count("title").sort(['count_title','name'],ascending = False).project(['name', 'count_title']).head(10)

,name,count_title
608,Zack Akerman,5
265,Jennifer (III) Lawrence,4
142,Dean Cain,4
47,Ben Stiller,4
476,Ranbir Kapoor,3
417,Nagarjuna Akkineni,3
381,Melissa Leo,3
264,Jeff Bridges,3
221,Ian McKellen,3
158,Eddie Redmayne,3


In [ ]:
sol_cp4r(release_date,moviecast,_)

In [33]:
%%sql

select moviecast.name, count(release_date.title) from moviecast natural join release_date
where n = 1 and release_date.year >= 2010 and (month = 11 or month = 12) and country = 'USA'
group by name order by count(release_date.title) desc limit(10);

10 rows affected.


name,count
Zack Akerman,5
Jennifer (III) Lawrence,4
Ben Stiller,4
Dean Cain,4
Ranbir Kapoor,3
Jeff Bridges,3
Christian Bale,3
Melissa Leo,3
Ian McKellen,3
Eddie Redmayne,3


In [ ]:
sol_cp4s(_)

## 5.  Show the title of all of the movies that were released in Germany before they were released in the USA in the year 2014.

In [39]:
release_date.query("(country == 'Germany' | country == 'USA') & year == 2014").\
cartesian_product(release_date.query("(country == 'Germany' | country == 'USA') & year == 2014")).\
    query('date_x < date_y & country_x == "Germany" & country_y == "USA" & title_x == title_y').project(['title_x'])

,title_x
42726,300: Rise of an Empire
83742,A Little Chaos
99123,A Million Ways to Die in the West
357182,Big Game
370854,Billy Elliot the Musical Live
396489,Blended
430669,Boyhood
488775,Captain America: The Winter Soldier
540045,Clouds of Sils Maria
685310,Der 7bte Zwerg


In [ ]:
sol_cp5r(release_date,moviecast,_)


In [40]:
%%sql

select x.title
from release_date x, release_date y
where x.country = 'Germany' and y.country = 'USA'
and x.date < y.date and x.title = y.title
and x.year = 2014 and y.year = 2014;

59 rows affected.


title
Rio 2
RoboCop
Serena
She's Funny That Way
Step Up All In
Taken 3
The Amazing Spider-Man 2
The Face of an Angel
The Grand Budapest Hotel
The Hobbit: The Battle of the Five Armies


In [41]:
sol_cp5s(_)